In [1]:
%load_ext autoreload
%autoreload 2

from meteo_lib import PROJECT  # または: from meteo_lib.paths import PROJECT
#out_dir  = PROJECT / "outputs"

from meteo_lib.meteo_lib import getBlock, get_HourlyData, get_10minData
from meteo_lib.my_path_utils import DATA, OUT

import pandas as pd
import math
import numpy as np
from datetime import datetime, timedelta, date, time
from metpy.units import units
from metpy.calc import dewpoint_from_relative_humidity
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams["font.family"] = "Hiragino Sans"   # 例：Noto Sans CJK JP でも可
mpl.rcParams["axes.unicode_minus"] = False    # －が豆腐になるのを回避

import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter, MultipleLocator
import os
from pathlib import Path
import calendar
from IPython.display import display 

In [2]:
# ---- BOM式 Apparent Temperature ----
def apparent_temperature(T, V, e=None, RH=None):
    if e is None:
        if RH is None:
            return pd.Series(np.nan, index=T.index)
        e = 6.105 * np.exp((17.27*T)/(237.7+T)) * (RH/100.0)  # hPa
    return T + 0.33*e - 0.70*V - 4.00


In [3]:
# Apparent Temperature の中央値を求める
def get_AT_median(s: pd.Series, t_min: pd.Timestamp, t_max: pd.Timestamp):
    s = pd.to_numeric(s, errors='coerce')
    idx = pd.to_datetime(s.index, errors='coerce')
    s = s.loc[idx.notna()]
    m = (s.index >= t_min) & (s.index <= t_max)
#    print(s.loc[m])
    return float(s.loc[m].median(skipna=True)) if m.any() else None

In [14]:
def DataConversion(df: pd.DataFrame, add_column: bool = False) -> pd.DataFrame:
    # 1) 日時→DatetimeIndex
    if '日時' in df.columns:
        df['日時'] = pd.to_datetime(df['日時'], errors='coerce')
        df = df.set_index('日時').sort_index()
    else:
        df.index = pd.to_datetime(df.index, errors='coerce')
    df = df.loc[df.index.notna()]

    # 2) 数値化
    for c in ['気温', '風速', '蒸気圧']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    # 3) 体感温度
    df['AT'] = apparent_temperature(df.get('気温'), df.get('風速'), df.get('蒸気圧'))

    # 4) 各日 06:00–18:00 に絞って日単位中央値
    df_daytime = df.between_time('06:00', '18:00')        # 全日付対象で日中のみ抽出
    at_daily = df_daytime['AT'].resample('D').median()     # 日ごと中央値（Series）
    temp_daily = df_daytime['気温'].resample('D').median()     # Series

    # ← ここで2列DataFrame化（列名を at_daily / temp_daily に統一）
    daily_df = pd.concat(
        [at_daily.rename('at_daily'), temp_daily.rename('temp_daily')],
        axis=1
    )
    
    if add_column:
        # 元の毎時DFの各行に「その日のAT中央値」を付ける
        df['at_daily'] = df.index.normalize().map(at_daily)
        df['temp_daily'] = df.index.normalize().map(temp_daily)
        return df  # 毎時データ＋列としてのAT中央値
    else:
        return daily_df  # 「1ヶ月分のAT日中央値」だけを返す


In [15]:
def get_Webdata(prec, block, y_start, y_end, m_start, m_end):
    frames = []
    for y in range(y_start, y_end+1):
        for m in range(m_start, m_end+1):
            days = calendar.monthrange(y, m)[1]
            for d in range(1, days+1):
                df = get_HourlyData(prec, block, y, m, d)
                AT_daily = DataConversion(df, add_column=False)
                frames.append(AT_daily)
    AT = pd.concat(frames, axis=0, ignore_index=False)
    display(AT)

    AT.sort_index(inplace=True)
    AT.to_csv(outdata)

In [ ]:
pref, station = '宮城県', '仙台'
info = getBlock(pref, station)
#print(info)
prec, block, st_name = info['prec'], info['block'], info["Name_2"]

y_start, y_end = 1970, 2024 
m_start, m_end = 1, 12

outdata = DATA / f"AT_{station}_{y_start}-{y_end}.csv"

if outdata.exists():
    print("from Local")
    AT = pd.read_csv(outdata)
else:
    print("from Web")
    AT = get_Webdata(prec, block, y_start, y_end, m_start, m_end)

from Web


In [ ]:
year = 1993
month_min, month_max = 1, 12
REF_YEAR = 2001
x_min, x_max =  pd.to_datetime(f"{year}-{month_min}-01"), pd.to_datetime(f"{year}-{month_max}-31")

def to_refdate(s: pd.Series) -> pd.Series:
    # 年を消して「月日だけ」を基準年に載せ替え（NaTはNaTのまま）
    return pd.to_datetime(
        np.where(s.notna(), s.dt.strftime(f"{REF_YEAR}" + "-%m-%d"), pd.NaT),
        errors="coerce"
    )
date_ref = pd.to_datetime(AT['日時'])
x = to_refdate(date_ref)

fig, ax1 = plt.subplots(figsize=(10, 5))
mask = date_ref.dt.year==year
ax1.plot(date_ref[mask], AT[mask]['temp_daily'], 
         linestyle='-', color="red", label=f'気温[{year}]')
ax1.plot(date_ref[mask], AT[mask]['at_daily'], 
         linestyle='-', color="blue", label=f'体感温度[{year}]')
ax1.set_xlim(x_min, x_max)
ax1.tick_params(axis='x', labelrotation=45)
ax1.set_ylim(-10,40)
ax1.grid()
ax1.legend()
plt.title(f'気温・体感温度 {station} {year}')
plt.show()

In [ ]:
year = 1993
x_min, x_max = pd.to_datetime(f"{year}-01-01"), pd.to_datetime(f"{year}-12-31")
y_min, y_max = -15, 10
fig, ax = plt.subplots(1,1,figsize=(10,5))
#ax.plot(AT.index, AT['at_daily'], linestyle='none', marker='o')
#ax.plot(AT.index, AT['temp_daily'], linestyle='none', marker='^')
#ax.plot(AT.index, AT['at_daily'], linestyle='-', label='AT')
#ax.plot(AT.index, AT['temp_daily'], linestyle='-', label='Temp')

ax.bar(AT.index, (AT['at_daily']-AT['temp_daily']), linestyle='-', label='f{year}')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
plt.title(f'「体感温度ー気温」 {station} {year}')
#ax.legend()
plt.savefig(OUT / f'体感温度ー気温{station}_{year}.png')
plt.show()

In [10]:
import seaborn as sns

def make_year_calendar_heatmap(
    df: pd.DataFrame,
    date_col: str,
    value_col: str,
    drop_feb29: bool = True,
    md_range: tuple[str, str] | None=None,
    cmap: str = "coolwarm",
    vmin: float | None=None,
    vmax: float | None=None,
    fig_size: tuple[float, float] = (12, 6),
):
    data = df[[date_col, value_col]].dropna(subset=[date_col]).copy()
    data[date_col] = pd.to_datetime(data[date_col], errors="coerce")
    data = data.loc[data[date_col].notna()]

    data["year"] = data[date_col].dt.year
    data["month"] = data[date_col].dt.month
    data["day"] = data[date_col].dt.day

    if drop_feb29:
        data = data[~((data["month"] == 2) & (data["day"] == 29))]

    REF_YEAR = 2001
    data["md_ref"] = pd.to_datetime(
        data[date_col].dt.strftime(f"{REF_YEAR}" + "-%m-%d"), errors="coerce"
    )

    # 月日範囲で抽出（同年内の範囲を想定）
    if md_range is not None:
        md_start = pd.to_datetime(f"{REF_YEAR}-{md_range[0]}")
        md_end   = pd.to_datetime(f"{REF_YEAR}-{md_range[1]}")
        data = data[(data["md_ref"] >= md_start) & (data["md_ref"] <= md_end)]

    # 透視図：index=年, columns=md_ref（昇順）
    table = (
        data.pivot_table(index="year", columns="md_ref", values=value_col, aggfunc="mean")
        .sort_index(axis=1)
        .sort_index(axis=0)
    )

    # プロット
    plt.figure(figsize=fig_size)
    # NaN色を調整（欠測が多い場合に灰色などに）

    vmin, vmax = -10, 40

    cmap_obj = mpl.colormaps.get_cmap(cmap).copy()
    cmap_obj.set_bad("lightgray")
    mask = table.lt(vmin) | table.gt(vmax)

    ax = sns.heatmap(
        table,
#        mask=mask,
#        cmap=cmap_obj,
        cmap="RdBu_r",
#        center=0,
        vmin=vmin, vmax=vmax,  # 年比較では固定推奨
        cbar_kws={"label": value_col},
    )

    # x軸：月初に目盛りを置く
    cols = table.columns
    # 列インデックスを必ず DatetimeIndex に揃える
    if not isinstance(cols, pd.DatetimeIndex):
        cols = pd.to_datetime(cols)
        # DataFrame本体の columns も揃えておくと後続も安全
        table.columns = cols

    # 月初（MS=Month Start）を作成
    REF_YEAR = 2001
    months = pd.date_range(f"{REF_YEAR}-01-01", f"{REF_YEAR}-12-01", freq="MS")
    #months = pd.to_datetime([f"{REF_YEAR}-{m:02d}-01" for m in range(1,13)])

    # DatetimeIndex.same-type で searchsorted（NumPyではなくPandasのIndexメソッド）
    month_locs = cols.searchsorted(months)
    # 表示範囲外を除外
    month_locs = month_locs[month_locs < len(cols)]

    ax.set_xticks(month_locs)
    # ラベル数は month_locs の長さに合わせる
    tick_labels = [m.strftime("%-m/%-d") for m in months[:len(month_locs)]]
    ax.set_xticklabels(tick_labels, rotation=0)

    ax.set_xlabel("Month / Day")
    ax.set_ylabel("Year")
    ax.set_title(f"Calendar Heatmap: {value_col}")

    plt.tight_layout()
    plt.show()


In [ ]:
# -------------------------
# 使い方サンプル
# -------------------------
# 例1) 年・月・日から date を作る場合（JMA風テーブル想定）
# df_raw に列 ['年','月','日','日平均気温'] があるとする
# df_raw = ...
# df_raw['date'] = pd.to_datetime(dict(year=df_raw['年'], month=df_raw['月'], day=df_raw['日']))
# make_year_calendar_heatmap(df_raw, date_col='date', value_col='日平均気温',
#                            drop_feb29=True, md_range=("06-01","09-30"),
#                            cmap="coolwarm", vmin=10, vmax=30, fig_size=(12,6))

# 例2) すでに 'date' と 'value' がある場合
#AT["日時"] = AT.index
AT['delta'] = AT['at_daily'] - AT['temp_daily']
print(AT)

"""
make_year_calendar_heatmap(AT, date_col='日時', value_col='delta',
                           drop_feb29=True, md_range=("06-01","10-31"),
                           cmap="mako", vmin=-5, vmax=5)
"""
make_year_calendar_heatmap(AT, date_col='日時', value_col='at_daily',
                           drop_feb29=True, md_range=("01-01","12-31"),
                           cmap="mako", vmin=-5, vmax=5)


In [ ]:
補足（実務のコツ）
色域は固定（vmin/vmax）：年間比較の“色の意味”がぶれません（例：気温なら vmin=10, vmax=30 など）。

うるう日の扱い：年比較を揃えるなら drop_feb29=True が無難。どうしても使いたい場合は 2/28 と 3/1 に分配・補間などの前処理を検討。

期間抽出：md_range=("06-01","09-30") で夏期などに絞れます。

欠測の色：cmap.set_bad("lightgray") で一目で分かるように。

列の粒度：日次前提。10分/1時間データなら pcolormesh（座標を渡す）を推奨。

列名が決まっていれば、そのまま差し替え済み版（仙台のJMA日平均気温など）も作れます。サンプルの列名を教えてください。